# The `Spectrum` class
This notebook demonstrates some of the methods of the `Spectrum` class.

In [34]:
# Imports
from SEDkit import FileSpectrum, Spectrum, Bandpass, Blackbody
import astropy.units as q
from astropy.io import fits
import astropy.table as at
from bokeh.io import output_notebook, show
from pkg_resources import resource_filename
import numpy as np
output_notebook()

Loading BokehJS ...

## Creating a `Spectrum`
A spectrum can be created from a file spectrum...

In [2]:
# Add a spectrum from a file, specifying the astropy units
file = resource_filename('SEDkit', 'data/L3_spectrum.fits')
s = FileSpectrum(file, wave_units=q.um, flux_units=q.W/q.m**2/q.um)

# Plot it
s.plot(draw=True)

...or `astropy.units.quantity.Quantity` arrays...

In [3]:
# Get the spectral data
wave, flux, unc = fits.getdata(file)
s = Spectrum(wave*q.um, flux*q.W/q.m**2/q.um, unc*q.W/q.m**2/q.um)

# Plot it
s.plot(draw=True)

# Resampling
Spectra can be resampled to a new resolution...

In [4]:
binned = s.resamp(resolution=100)
binned.plot(draw=True)

...or to a new wavelength array...

In [5]:
res = s.resamp(wave=np.linspace(1, 2, 200)*q.um)
res.plot(draw=True)

## Make a composite `Spectrum`
Spectra can be added together so that a new spectrum is returned, which takes the mean of the fluxes in overlapping wavelength regions.

In [6]:
# Make a Spectrum
file = resource_filename('SEDkit', 'data/L3_spectrum.fits')
s0 = FileSpectrum(file, wave_units=q.um, flux_units=q.W/q.m**2/q.um, name='Foobar')
s0.plot(draw=True)

# And another that looks different (downsampled, different wave units and values, randomized flux)
wave, flux, unc = s0.spectrum
wave += 2.01*q.um
wave = wave.to(q.AA)
flux = np.random.normal(flux, scale=np.max(unc))*flux.unit
s1 = Spectrum(wave[::4][50:-50], flux[::4][50:-50], unc[::4][50:-50])
s1.plot(draw=True)

In [7]:
# Add them together
s2 = s0+s1
fig = s2.plot()
fig = s1.plot(fig)
show(s0.plot(fig))

Good! The spectrum should be the mean flux in the overlapping wavelength regions.

## Normalizing a `Spectrum`
A `Spectrum` can be normalized to a magitude in a given bandpass...

In [30]:
# Create a bandpass object and specify the magnitude
bp = Bandpass('2MASS.J')
new = s.renormalize(11.3, bp)

# Normalize and plot both
fig = s.plot()
show(new.plot(fig))

...or a table of magnitudes...

In [31]:
# Make some dummy photometry
bands = ['2MASS.J','2MASS.H','2MASS.Ks']
mags = np.array([3.5e-18, 2.5e-18, 1e-18])*q.W/q.m**2/q.um
errs = np.array([3e-19, 2.5e-19, 2e-19])*q.W/q.m**2/q.um
bps = [Bandpass(b) for b in bands]
effs = np.array([i.eff.value for i in bps])*q.AA
photometry = at.QTable([bands, effs, mags, errs, bps], names=('band','eff','app_flux','app_flux_unc','bandpass'))

# Normalize to all magnitudes
norm_all = s.norm_to_mags(photometry)

# And get the synthetic fluxes for comparison
syn_mags = [norm_all.synthetic_flux(b)[0].value for b in bps]

# Plot it
fig = norm_all.plot()
fig.circle(effs.value, mags.value, color='black', size=15, legend='Input mags')
fig.circle(effs.value, syn_mags, color='red', size=15, legend='Synthetic mags')
show(fig)

...or another `Spectrum` object...

In [32]:
# As a nice visual example, we'll make another spectrum with the dame data but
# different wave units and values and 10x randomized flux
wave, flux, unc = s.spectrum
wave += 0.01*q.um
# wave = wave.to(q.AA)
flux = np.random.normal(flux, scale=np.max(unc))*flux.unit
smod = Spectrum(wave[::4][100:-400], flux[::4][100:-400]*10, unc[::4][100:-400])

# Plot the input spectra
fig = s.plot()
show(smod.plot(fig))

# Normalize and plot both
new = s.norm_to_spec(smod)
fig = smod.plot()
show(new.plot(fig))

## Creating a blackbody spectrum
A `Blackbody` is just a child class of `Spectrum`, which can be generated like so...

In [38]:
# Supply a wavelength axis and tempertaure
wave = np.linspace(0.5, 10, 1000)*q.um
bb = Blackbody(wave, 2345*q.K)

# Plot it
bb.plot(draw=True)

No uncertainty array for this spectrum. Using SNR= 10


Since it inherits from the `Spectrum` class, all methods discussed above apply. For example, let's normalize our spectrum to this blackbody...

In [40]:
# Normalize the spectrum to the blackbody
new = s.norm_to_spec(bb)

# Plot it
fig = bb.plot()
show(new.plot(fig))